In [1]:
import sys
import datetime
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

sys.path.append('..')

from definitions import INPUT_DATA_DIR, PARSED_DATA_DIR, SUB_DIR, FULL_DATA_DIR, INPUT_DATA_DIR, AUTHORS_DIR

%reload_ext autoreload
%autoreload 2

In [2]:
version = {
    'lightgbm': lgb.__version__,
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'scikit-learn': sk.__version__ 
}
print(version)

{'lightgbm': '3.3.2', 'pandas': '1.4.3', 'numpy': '1.23.1', 'scikit-learn': '1.1.1'}


# 1. Загрузка данных

In [3]:
from src.transformers.base import Compose
from src.transformers.preprocess import (
    LoaderMergePreprocess, 
    CategoryFromTextPreprocess, 
    AuthorsPreprocess,
    TagsPreprocess,
    FeaturePreprocess,
    SaverPreprocess,
    NatashaTransformer
)

In [4]:
Preprocessor = Compose(
    transforms=[
        LoaderMergePreprocess(name='loading'),
        CategoryFromTextPreprocess(name='category from text'),
        AuthorsPreprocess(name='authors preprocess'),
        TagsPreprocess(name='tags preprocess'),
        FeaturePreprocess(name='feature selector'),
        NatashaTransformer(name='natasha name entity'),
        SaverPreprocess(name='saving files')
    ]
)

Данный код запускет `preprocessing` данных

In [8]:
train = Preprocessor(data=pd.DataFrame(), mode='train')
test = Preprocessor(data=pd.DataFrame(), mode='test')

test: 100%|████████████████████████████████████████████████████████████████████████| 3000/3000 [00:52<00:00, 57.53it/s]


Либо можно их сразу загрузить

In [5]:
train = pd.read_json(FULL_DATA_DIR / 'full_train.json')
test = pd.read_json(FULL_DATA_DIR / 'full_test.json')

In [6]:
train['NAMES_FROM_TITLE'] = (
    train['ORG_TITLE'].apply(lambda x: x if isinstance(x, list) else []) + 
    train['LOC_TITLE'].apply(lambda x: x if isinstance(x, list) else []) + 
    train['PER_TITLE'].apply(lambda x: x if isinstance(x, list) else [])
)
test['NAMES_FROM_TITLE'] = (
    test['ORG_TITLE'].apply(lambda x: x if isinstance(x, list) else []) + 
    test['LOC_TITLE'].apply(lambda x: x if isinstance(x, list) else []) + 
    test['PER_TITLE'].apply(lambda x: x if isinstance(x, list) else [])
)

## 2. Предсказание для существующих

In [7]:
from src.prediction.utils import predict_for_exist
from src.constants import FeaturesConstants

In [8]:
exist_prediction = predict_for_exist(train, test)
train_and_tests_ctr_ = list(set(train['page_id']) & set(test['page_id']))

In [9]:
train_data = (
    train[~(train['page_id'].isin(train_and_tests_ctr_))]
    .drop_duplicates(FeaturesConstants.target + ['document_id', 'page_id', 'ctr'])
)
train_data = train_data[train_data['full_reads_percent'] <= 100]
train_data = train_data[pd.to_datetime(train_data['publish_date']).dt.year >= 2022]
train_data = train_data[train_data['views'] <= 1000000].copy()

test_data = test[~(test['page_id'].isin(train_and_tests_ctr_))].copy()

In [10]:
y_data = train_data[FeaturesConstants.target].copy()

In [11]:
train_data.shape, test_data.shape

((6912, 37), (2967, 34))

## 3. Feature Generation

## 3.1 Identifier

- `page_id`: `624ac09c9a7947db3d80c98e`

## 3.2 Date

- `publish_date`: `2022-04-04 10:29:44` + `timedelta(hours=3)`

## 3.3 Title

- `title` - title новости от организаторов, (**текст**)
- `title_parsed` - спарсенный title новости, (**текст**)
- `optional[title_parsed_from_yandex]` - заголовок статьи в `yandex`, (**текст**)
- `title_preprocessed` - удалены `category`

## 3.4 Category

- `category` - категория новости, **hash**
- `category_parsed` - спарсенная категория новости, **текст**
- `category_from_title` - категория, выделенная из текста, **Optional[текст]**

## 3.5 Ctr

- `ctr` - показатель кликабельности, **Optional[float64]**

## 3.6 Tags

- `tags` - таги, закодированные от организаторов, **Optional[hash]**
- `tags_parsed` - спарсенные тэги, **Optional[текст]**

## 3.7 Auhtors

- `authors` - авторы новости от организаторов, **Optional[hash]**
- `authors_parsed` - авторы новости **Optional[текст]**

## 3.8 Text

- `news_text_parsed` - текст новости, полный, **текст** 
- `news_text_overview_parsed` - некоторая выжимка новости, **текст**
- `news_amount_of_paragraphs_parsed` - количество параграфов в тексте, **int64**
- `news_amount_of_inline_items_parsed` - количество ссылок на другие новости в тексте новости, **int64**
- `news_inline_titles_parsed` - заголовки на другие статьи в тексте новости, **Optional[текст]**
- `news_has_image_parsed` - есть ли в новости картинка, **int64**
- `news_image_title_parsed` - подпись к изображению, если есть, **Optional[текст]**

## 3.9. Features Selector Transformer

In [12]:
from src.transformers.feature_generation import (
    DatetimeTransformer,
    TitleTransformer,
    CategoryTransformer,
    CTRTransformer,
    TagsTransformer,
    AuthorsTransformer,
    TextTransformer,
    NatashaTextTransformer,
    FeatureSelector,
    TfidfVectorTransformer,
    CountTfIdfVectorizer,
    MultiLabelTransformer
)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shiro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 4. Model Fitting

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import optuna

from sklearn.model_selection import RepeatedKFold
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer,
)

import warnings
warnings.filterwarnings("ignore")

C:\Users\shiro\Desktop\DATABASE\competitions\summer_2022\rbk\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def plotImp(
    model,
    X, 
    num=10, 
    fig_size = (40, 20)
):
    feature_imp = pd.DataFrame(
        {'Value': model.feature_importances_,
         'Feature': X.columns}
    )
    fig, ax = plt.subplots(1, 1, figsize=fig_size)
    sns.barplot(
        x="Value", 
        y="Feature", 
        data=feature_imp.sort_values(by="Value", ascending=False)[0:num],
        ax=ax
    )
    plt.show()
    


In [15]:
kfold = KFold(n_splits=3, shuffle=True, random_state=239)

In [16]:
result_dict = {
    'views': {'r2': [], 'trees': [], 'prediction': []},
    'full_reads_percent': {'r2': [], 'trees': [], 'prediction': []},
    'depth': {'r2': [], 'trees': [], 'prediction': []}
}
for target_variable in FeaturesConstants.target:
    FeatureGenerator = Compose(
        transforms=[
            DatetimeTransformer(
                target_variable=target_variable
            ),
            TitleTransformer(),
            CTRTransformer(
                eps=(
                    0.1 
                    if target_variable == 'views' 
                    else 0.5
                )
            ),
            TagsTransformer(target_variable=target_variable),
            AuthorsTransformer(
                target_variable=target_variable
            ),
            TextTransformer(
                use_text_transformer=(
                    target_variable 
                    if target_variable == 'full_reads_percent' 
                    else None
                )
            ),
            NatashaTextTransformer(),
            CategoryTransformer(),
            FeatureSelector()
        ]
    )
    train_features = FeatureGenerator(
        data=train_data, 
        mode='train'
    )
    test_features = FeatureGenerator(
        data=test_data, 
        mode='test'
    )
    if target_variable == 'views':
        tfidf = TfidfVectorTransformer(
            col='NAMES_FROM_TITLE',
            tfidf_vectorizer=TfidfVectorizer(
                ngram_range=(1, 4),
                min_df=1,
                max_features=128
            )
        )
        tfidf.fit(train_data)
        train_dd = tfidf.transform(train_data)
        test_dd = tfidf.transform(test_data)
        train_features = pd.concat([train_features, train_dd], axis=1)
        test_features = pd.concat([test_features, test_dd], axis=1)
    
    if target_variable == 'depth':
        tfidf = TfidfVectorTransformer(
            col='NAMES_FROM_TITLE',
            tfidf_vectorizer=TfidfVectorizer(
                ngram_range=(1, 4),
                min_df=1,
                max_features=128
            )
        )
        tfidf.fit(train_data)
        train_dd = tfidf.transform(train_data)
        test_dd = tfidf.transform(test_data)
        train_features = pd.concat([train_features, train_dd], axis=1)
        test_features = pd.concat([test_features, test_dd], axis=1)
    if target_variable == 'full_reads_percent':
        m = MultiLabelTransformer(col='authors_parsed')
        m.fit(train_data)
        train_author = m.transform(train_data)
        test_author = m.transform(test_data)
        train_features = pd.concat(
            [train_features, train_author], 
            axis=1
        )
        test_features = pd.concat(
            [test_features, test_author], 
            axis=1
        )
        
    if target_variable != 'full_reads_percent':
        tfidf = TfidfVectorTransformer(
            col='tags_parsed',
            tfidf_vectorizer=TfidfVectorizer(
                ngram_range=(1, 4) if target_variable != 'full_reads_percent' else (1, 4),
                max_features=256 if target_variable != 'full_reads_percent' else 256,
                min_df=5 if target_variable != 'full_reads_percent' else 5
            )
        )
        tfidf.fit(train_data)
        train_dd = tfidf.transform(train_data)
        test_dd = tfidf.transform(test_data)
        train_features = pd.concat([train_features, train_dd], axis=1)
        test_features = pd.concat([test_features, test_dd], axis=1)
        
        cec = CountTfIdfVectorizer(
            col='authors_parsed',
            count_vectorizer=CountVectorizer(
                ngram_range=(2, 2),
                min_df=2
            )
        )
        cec.fit(train_data)
        train_features['author_popularity'] = cec.transform(train_data)
        test_features['author_popularity'] = cec.transform(test_data)

    for train_index, val_index in kfold.split(train_features):
        X_train, X_valid = (
            train_features.reset_index(drop=True).iloc[train_index, :], 
            train_features.reset_index(drop=True).iloc[val_index, :]
        )
        y_train, y_valid = (
            y_data[target_variable].reset_index(drop=True).iloc[train_index], 
            y_data[target_variable].reset_index(drop=True).iloc[val_index]
        )
        if target_variable == 'views':
            y_train = np.log(y_train)
            y_valid = np.log(y_valid)
        elif target_variable == 'depth':
            y_train = np.log(np.log(y_train))
            y_valid = np.log(np.log(y_valid))
        else:
            y_train = np.log(y_train)
            y_valid = np.log(y_valid)
                    
        lgbm_regressor = lgb.LGBMRegressor(
            objective='regression',
            random_state=33,
            early_stopping_round=100, 
            n_estimators=500,
            subsample=1,
            colsample_bytree=0.75,
            learning_rate=(
                0.08 
                if target_variable == 'full_reads_percent' 
                else 0.08
            ),
            max_depth=-1,
            verbose=-1
        )
        lgbm_regressor.fit(
            X_train, 
            y_train, 
            eval_set=[(X_valid, y_valid)], 
            eval_metric='r2', 
            verbose=-1
        )

        y_pred = lgbm_regressor.predict(X_valid)
        r2 = r2_score(y_valid, y_pred)
        result_dict[target_variable]['r2'].append(r2)
        result_dict[target_variable]['trees'].append(lgbm_regressor.booster_.trees_to_dataframe())

        
        pro = lgbm_regressor.predict(test_features)
        if target_variable == 'views':
            pro = np.exp(pro)
        elif target_variable == 'depth':
            pro = np.exp(np.exp(pro))
        else:
            pro = np.exp(pro)
        assert len(pro[pro < 0]) == 0
        result_dict[target_variable]['prediction'].append(pro)
    print(f'{target_variable}: ', np.mean(result_dict[target_variable]['r2']))

full_score = (
    np.mean(result_dict['views']['r2']) * 0.4 + 
    np.mean(result_dict['full_reads_percent']['r2']) * 0.3 + 
    np.mean(result_dict['depth']['r2']) * 0.3
)
full_score_2 = (
    np.median(result_dict['views']['r2']) * 0.4 + 
    np.median(result_dict['full_reads_percent']['r2']) * 0.3 + 
    np.median(result_dict['depth']['r2']) * 0.3
)
print(f'Final score: {full_score}, median: {full_score_2}')

views:  0.7155297482884211
depth:  0.8440000609380154
full_reads_percent:  0.5393429928349939
Final score: 0.7012148154472712, median: 0.7018418003000428


# Sub4 - best

In [19]:
sub = pd.DataFrame(
    {
        'document_id': test_data['document_id'].values,
        'views': pd.DataFrame(result_dict['views']['prediction']).T.mean(axis=1),
        'depth': pd.DataFrame(result_dict['depth']['prediction']).T.mean(axis=1),
        'full_reads_percent': pd.DataFrame(result_dict['full_reads_percent']['prediction']).T.mean(axis=1),
    },
)
sub.index = test_data.index

In [20]:
sub_full = pd.concat((sub, exist_prediction)).sort_index()

In [21]:
ss = pd.read_csv('../sub/sample_solution.csv')

In [22]:
sub_full_2 = pd.DataFrame(ss['document_id'])

In [23]:
views = []
full_reads_percent = []
depth = []
for j, i in sub_full_2.iterrows():
    dummy = sub_full[sub_full['document_id'] == i['document_id']]
    views.append(dummy['views'].values[0])
    full_reads_percent.append(dummy['full_reads_percent'].values[0])
    depth.append(dummy['depth'].values[0])

In [24]:
sub_full_2['views'] = views
sub_full_2['full_reads_percent'] = full_reads_percent
sub_full_2['depth'] = depth

In [25]:
sub_full_2.to_csv(SUB_DIR / 'last3.csv', index=False)